In [1]:
# Answering question, has security update posture improved over time?

import pandas as pd
import numpy as np
import researchpy # pip install researchpy
import statistics
import re 
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.io as pio
%config InlineBackend.figure_format = 'svg'

carrierDF=pd.read_csv('../data/allCarrierData_final.csv')
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("Samgung","Samsung")
carrierDF.difference=carrierDF.difference.astype('float')
print(carrierDF.head())

  Carrier Manufacture               Model Release_Date Bulletin_Level  \
0    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-04-01   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18     2018-10-01   
4    AT&T  BlackBerry   BlackBerry KEYone   2017-09-26     2017-09-01   

   difference Android_Level_Guessed  \
0        61.0                 False   
1         2.0                  True   
2        84.0                 False   
3        78.0                 False   
4        25.0                 False   

                                               build  Year_Phone_Release  \
0        JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0                2017   
1                                nan YB4UA40File nan                2017   
2        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0                2017   
3        JO.2.0.c1-002

In [2]:
# Number of bulletins per year for a phone
carrierDF=carrierDF[carrierDF.Carrier!="Verizon"]
allYears=carrierDF.Year_Phone_Release.unique()
allYears=np.sort(allYears)
allYears=np.delete(allYears,0)
allYears=np.delete(allYears,0)
allYears=np.delete(allYears,0)
print(allYears)
carrierDF['Bulletin_Level_Year']=carrierDF.Bulletin_Level.apply(lambda x: str(x)[:4])
carrierDF['Release_Year']=carrierDF.Bulletin_Level.apply(lambda x: str(x)[:4])
carrierDF['Bulletin_Level_Year']=carrierDF['Bulletin_Level_Year'].astype(int)
carrierDF['Release_Year']=carrierDF['Release_Year'].astype(int)

perRelease={}
for year in allYears:
    perRelease[year]=[]
    
for index,year in enumerate(allYears):
    print(year)
    tmp=carrierDF[carrierDF.Year_Phone_Release==year] # for each phone model released in proper year
    tmp=tmp[tmp.Bulletin_Level_Year==year] # all bulletins applied in same year (treating as original)
    tmp=tmp[tmp.Release_Year==year] # only looking at true updates (remove all updates released outside the year)
    print("Total Updates: "+str(len(tmp)))
    print("Unique Devices: "+str(len(tmp.Model.unique()))) 
    print("      Average Update per Device: "+str(len(tmp)/len(tmp.Model.unique()))[:5])
    print("Unique Bulletins: "+str(len(tmp.Bulletin_Level.unique())))  
    perRelease[year]=tmp.difference.values
    print(perRelease[year][:5])
    print("------------------------------------------------------------------------")


[2016 2017 2018 2019]
2016
Total Updates: 58
Unique Devices: 19
      Average Update per Device: 3.052
Unique Bulletins: 10
[74. 13. 46. 89.  4.]
------------------------------------------------------------------------
2017
Total Updates: 92
Unique Devices: 19
      Average Update per Device: 4.842
Unique Bulletins: 11
[25. 30. 14. 26. 21.]
------------------------------------------------------------------------
2018
Total Updates: 102
Unique Devices: 30
      Average Update per Device: 3.4
Unique Bulletins: 12
[101.  85.  24.  26.  25.]
------------------------------------------------------------------------
2019
Total Updates: 70
Unique Devices: 22
      Average Update per Device: 3.181
Unique Bulletins: 12
[16.  7. 16. 24. 18.]
------------------------------------------------------------------------


In [3]:
data=[]
for yr in allYears:
        print(yr)
        print(statistics.mean(perRelease[yr]))
        print()
        data.append(go.Box(
                y=perRelease[yr],
                name=str(yr),
            boxpoints = 'all',
            ))
        
# Edit the layout
layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Update Latency (days)',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Year Phone Released'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-since-release-years.png')

2016
21.517241379310345

2017
20.456521739130434

2018
25.519607843137255

2019
19.228571428571428



In [4]:
# Per bulletin, look at the difference box plot
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
uniqueBulletins=carrierDF.Bulletin_Level.unique()
uniqueBulletins.sort()
print("Unique number of bulletins: ")
print(len(uniqueBulletins))
print()

# print(uniqueBulletins)
carrierDF['Bulletin_Level_Year']=carrierDF['Bulletin_Level_Year'].astype(int)


data=[]
data2=[]
vals2=[[],[]]
vals3=[[],[]]
for bul in uniqueBulletins:
    tmp=carrierDF[carrierDF.Bulletin_Level==bul]
    bulTime=pd.to_datetime(str(bul))
    tempX=[[bulTime.strftime("%Y") for _ in tmp.difference.values]]
    tempX.append([bulTime.strftime("%m") for _ in tmp.difference.values])
    data.append(go.Box(
            #x=tempX.copy(), #[bulTime.strftime("%Y-%m") for _ in tmp.difference.values],
            y=tmp.difference.values,
            name=bulTime.strftime("%m/%Y"),#bulTime.strftime("%Y-%m"), #str(bul)
            showlegend=False,
            marker=dict(color="rgb(31, 119, 180)")
            #boxpoints = 'all',
        ))
    
#     vals2[0].append(str(bul))
#     vals2[1].append(len(tmp))
    
#     vals3[0].append(str(bul))
#     vals3[1].append(statistics.mean(tmp.difference.values))
    
# data2.append(go.Scatter(
#             x=vals2[0],
#             y=vals2[1],
#             name="Number of Updates",
#             #boxpoints = 'all',
#             line_color='deepskyblue'
#         ))

# data2.append(go.Scatter(
#         x=vals3[0],
#         y=vals3[1],
#         name="Average Latency",
#         #boxpoints = 'all',
#         line_color='dimgray'
#     ))
        
# Edit the layout

layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               xaxis = dict(title = 'Android Security Bulletin',ticks="inside",tickwidth=2, tickcolor='rgb(219, 219, 219)',ticklen=15),
              xaxis = dict(title = 'Android Security Bulletin',type="category",tickmode="array",tickformat = '%m/%Y',
                                tickvals = [0,12, 24, 36, 48], # 52 for last section
                                ticktext = ["08/2015",'08/2016', '08/2017', '08/2018', '08/2019'], # '12/2019'
                                ticks="outside",
                                #type="multicategory"
                          ), #
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20),
              )


fig = dict(data=data, layout=layout)
# py.offline.iplot(fig, filename='avg-latency-since-release-years.png')

pio.write_image(fig, 'latency-across-time.pdf', width=1100, height=600)


# # Graph to display number of points
# layout = dict(#title = "Average Latency per 6 Month Periods after Release",
#               yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
#               xaxis = dict(title = 'Android Security Bulletins'),
#               plot_bgcolor='rgba(0,0,0,0)',
#               )
# fig = dict(data=data2, layout=layout)
# py.offline.iplot(fig, filename='avg-latency-since-release-years.png')



####################################################################################
####################################################################################
####################################################################################
####################################################################################



# Per bulletin, look at the difference box plot
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
uniqueBulletins=carrierDF.Bulletin_Level.unique()
uniqueBulletins.sort()
print("Unique number of bulletins: ")
print(len(uniqueBulletins))
print()

# print(uniqueBulletins)
carrierDF['Bulletin_Level_Year']=carrierDF['Bulletin_Level_Year'].astype(int)


data=[]
data2=[]
vals2=[[],[]]
vals3=[[],[]]
vals4=[[],[]]
for bul in uniqueBulletins:
    tmp=carrierDF[carrierDF.Bulletin_Level==bul]
    bulTime=pd.to_datetime(str(bul))
    tempX=[[bulTime.strftime("%Y") for _ in tmp.difference.values]]
    tempX.append([bulTime.strftime("%m") for _ in tmp.difference.values])
    
    vals2[0].append(str(bul))
    vals2[1].append(len(tmp))
    
    vals3[0].append(str(bul))
    vals3[1].append(statistics.mean(tmp.difference.values))
    
    vals4[0].append(str(bul))
    vals4[1].append(statistics.median(tmp.difference.values))
    
data2.append(go.Scatter(
            x=vals2[0],
            y=vals2[1],
            name="Number of Updates",
            #boxpoints = 'all',
            line_color='deepskyblue'
        ))

data2.append(go.Scatter(
        x=vals3[0],
        y=vals3[1],
        name="Average Latency",
        #boxpoints = 'all',
        line_color='dimgray',
    ))

data2.append(go.Scatter(
        x=vals4[0],
        y=vals4[1],
        name="Median Latency",
        #boxpoints = 'all',
        line_color='olive'
    ))


# Graph to display number of points
layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Average Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Android Security Bulletins'),
              plot_bgcolor='rgba(0,0,0,0)',
              )
fig = dict(data=data2, layout=layout)
py.offline.iplot(fig, filename='avg-latency-since-release-years.png')





Unique number of bulletins: 
53

Unique number of bulletins: 
53



In [5]:
# Per bulletin, look at the difference box plot
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
uniqueBulletins=carrierDF.Bulletin_Level.unique()
uniqueBulletins.sort()



# For each bulletin, only consider updates from phones within only 3 years
#indices=(df['birth_date'] > start_date) & (df['birth_date'] <= end_date)
carrierDF['Bulletin_Level_3ago']=carrierDF['Bulletin_Level'] - pd.DateOffset(years=3) # 3 years since bulletin released
carrierDF['Phone_Released']= carrierDF.Year_Phone_Release.map(str) + "/" +carrierDF.Month_Phone_Release.map(str)
carrierDF['Phone_Released']=carrierDF['Phone_Released'].apply(lambda x: re.sub("/([0-9])$",r'/0\1/01',x))
carrierDF.Phone_Released=pd.to_datetime(carrierDF.Phone_Released)
carrierDF_filtered3yr=carrierDF[(carrierDF.Phone_Released>carrierDF.Bulletin_Level_3ago)] # phones can only be released 3 years prior to the bulletin
# checkMe=carrierDF_filtered3yr[carrierDF_filtered3yr.Bulletin_Level=='2019-10-01T00:00:00.000000000']
# print(checkMe.Phone_Released.unique())
data=[]
data2=[]
vals2=[[],[]]
vals3=[[],[]]
for bul in uniqueBulletins:
    tmp=carrierDF_filtered3yr[carrierDF_filtered3yr.Bulletin_Level==bul]
    
    data.append(go.Box(
            y=tmp.difference.values,
            name=str(bul),
            #boxpoints = 'all',
            showlegend=False,
        ))
    
    vals2[0].append(str(bul))
    vals2[1].append(len(tmp))
    
    vals3[0].append(str(bul))
    vals3[1].append(statistics.mean(tmp.difference.values))
    
data2.append(go.Scatter(
            x=vals2[0],
            y=vals2[1],
            name="Number of Updates",
            #boxpoints = 'all',
            line_color='deepskyblue'
        ))

data2.append(go.Scatter(
        x=vals3[0],
        y=vals3[1],
        name="Average Latency",
        #boxpoints = 'all',
        line_color='dimgray'
    ))
        
# Edit the layout
layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Update Latency (days)',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Android Security Bulletins'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-since-release-years.png')


# Graph to display number of points
layout = dict(#title = "Average Latency per 6 Month Periods after Release",
              yaxis = dict(title = 'Number of Updates',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Android Security Bulletins'),
              plot_bgcolor='rgba(0,0,0,0)',
              
              )
fig = dict(data=data2, layout=layout)
py.offline.iplot(fig, filename='avg-latency-since-release-years.png')



